In [6]:
# import  
import pandas as pd
import numpy as np

# Import functions to scale, compute accuracy and split data

from sklearn.preprocessing import scale

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.model_selection import train_test_split

# Import models, including BaggingClassifier meta-model

from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

# Set seed for reproducibility
SEED =1


# read training data to a DataFrame df
df = pd.read_csv('train_titanic.csv')


# Preprocessing


# drop misssing 'Embarked' values
df = df.dropna(subset=['Embarked'])

# fill in missing 'Age values'
median_value = df['Age'].median()
df['Age'] = df['Age'].fillna(median_value)

# convert 'Sex' column type to category
df['Sex'] = df['Sex'].astype('category')

# Dummy variable encoding
dummies = pd.get_dummies(df[['Sex']], prefix_sep = '_')

# add new binary 'Female' column to df for numerical analysis
#df = pd.concat([df, dummies], axis = 1)
df['Female'] = dummies['Sex_female']

# convert 'Embarked' column type to category
df['Embarked'] = df['Embarked'].astype('category')

# Dummy variable encoding
dummies = pd.get_dummies(df[['Embarked']], prefix_sep = '_')

# add new binary 'Embarked' columns to df for numerical analysis
df['Embarked_C'] = dummies['Embarked_C']
df['Embarked_S'] = dummies['Embarked_S']



# read numerical columns of df into Numpy arrays in order to feed into model

X = df.drop(['Survived', 'Name', 'Ticket', 'Cabin', 'Embarked', 'Sex'], axis = 1).values
y = df['Survived'].values


X = scale(X) # same result without scaling

# Split data into 70% train and 30% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, 
                                                    random_state = SEED, 
                                                    stratify = y)

# Instantiate classifier

dt = DecisionTreeClassifier(max_depth= 3, max_features= 0.6, 
                            min_samples_leaf= 0.04, random_state=SEED)


# Instantiate a BaggingClassifier 'bc'

bc = BaggingClassifier(base_estimator = dt, n_estimators = 300, n_jobs = -1)

# Fit 'bc' to the traing set and predict test set labels
bc.fit(X_train, y_train)
y_pred = bc.predict(X_test)

# Evaluate the test-set accuracy of 'bc'
print('Bagging Classifier test set accuracy: {:.3f}'.format(accuracy_score(y_test, y_pred)))

print('5 fold cross validation scores =', cross_val_score(bc, X, y, cv = 5))

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


Bagging Classifier test set accuracy: 0.816
5 fold cross validation scores = [0.76966292 0.79213483 0.81460674 0.7752809  0.80225989]
[[154  11]
 [ 38  64]]
              precision    recall  f1-score   support

           0       0.80      0.93      0.86       165
           1       0.85      0.63      0.72       102

    accuracy                           0.82       267
   macro avg       0.83      0.78      0.79       267
weighted avg       0.82      0.82      0.81       267

